In [70]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [91]:
#API key
api_key = "RGAPI-3d5dd86c-4fd4-4025-80b0-0077d0eb3f00" #API key
api_key2 = "RGAPI-a8a3d8ab-d224-4f1c-9e77-126dac5b6473"
api_key3 = "RGAPI-f3139655-6585-4874-a406-4cadfadb1a5e"


In [107]:
"""_summary_
    This function returns the summonerId of master + ranks players in europe west.
    
    _description_
    This function returns the summonerId of the player by using the API key.
    
    _parameters_
    api_key : str
        The API key that is used to authenticate the request
"""
    
def get_summonerId(api_key):
    challenger_url = 'https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Challanger ENDPOINT
    grandmaster_url = 'https://euw1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Grandmaster EDNPOINT
    master_url = 'https://euw1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Master ENDPOINT

    league_url_lists = [challenger_url, grandmaster_url, master_url] # Creating a list with the 3 endpoints (master, grandmaster and challenger)

    df_list = [] # A list that will store the response from the get request (league_df)

    for url in league_url_lists: # Loop interation through league_url_lists
        resp = requests.get(url) # Sending a get request to the url
        league_df = pd.json_normalize(resp.json()['entries'])
        df_list.append(league_df)
    
    final_league_df = pd.concat(df_list) # Concatenating the stored information in league_url_lists to create the final_league_df

    return final_league_df

In [108]:
league_df = get_summonerId(api_key)
print('league_df has information about {} players between master and challenger elo'.format(league_df.shape[0]))

league_df has information about 17003 players between master and challenger elo


In [109]:
def get_summonerinfo_by_summonerId(df, api_key):
    
    summoner_list = [] # list that will store the each summoner_info DataFrame

    for i in range(len(df)): # len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        # Creating a dynamic api_ulr for each register in the summonerId column using iloc
        api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)

        # Repeating the process we did at the begining
        resp = requests.get(api_url)
        print(resp.status_code)

        # if, elif and while true to deal with riot api rate limit
        if resp.status_code == 200:
            pass
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 10 second') #approximate 10 second wait before trying again
                    time.sleep(10)

                    # Trying again
                    api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break

        summoner_info = pd.json_normalize(resp.json()) # The same process we did at the beggining to create the DataFrame
        summoner_list.append(summoner_info) # Appending the DataFrame generated (summoner_info) into the list "summoner_list"
        
    final_summoner_info = pd.concat(summoner_list) # Using pd.concat to concatenate the results stored in the list "summoner_list"
            
    return final_summoner_info
    

In [110]:
summoner_df = get_summonerinfo_by_summonerId(league_df[:10], api_key)

200
200
200
200
200
200
200
200
200
200


In [111]:
summoner_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,yOZI1FSAt9dN8DS2CQKygBaoGrF2MjjyVkgzOJtrtxnk1tcr,0KFai1ESdX-ELZCOJGCpr2KD8B9Ez6_Cb9mMGCGi__dAC-...,pa2Pay4bZiLbFRCsFGv3PIVUyhkMkGvs01BMXnhZYCsb-i...,TTV NattyNattLoL,3592,1710427020778,859
0,olbE76G7T-Lph3LKtqEspWawaB7b0RTwkulbe7OPd7O1RbQ,GifW-kYwUK2QUK3VprksWtNabDGcRWuhKiaLdgv-8VFhGsY,EzLXDeUKWR55HJ04BcPniwMEpmSuXbi8GylwLZUSdn4ahs...,TakeSet sama,7,1710366406896,834
0,cLhAtVgqpQRLr9S4bw7z1qMUnuGOJl5wvMoMi_qeik1Hk7A,LF5FpBL_DmSyhI0rnrHEyVT6DLzKMbnh5VmRDPWxPDw0GQ,4tL4xiXd3viFZn7JVk2CxXpfvlvo7_ICT7AC_4U-zShNt1...,SLY Wakz,3456,1710445371089,805
0,BZyTrhEOww4Gqzz3za9jYfcEckYBk3Q7ymMYBeq6Tvr0B68,NxghRUvG-10N7vFweRvRrK99PeyQYbPji1aq6OIBY_kGl6c,-DyBYHx3xeMAnTkia_zT-8bYQ1_MPa-xJqsUXeVBptGXGh...,FXN Trailblazer,3614,1710440963505,1153
0,8zvvmSE8T2ExGGVu2bYgsVBpn2gY69nOb3WldIIfhbJbdlg,B_1sg_3QIaRNNYUqpYRWAKlrfYlBHsNPNRaM-y1EM2eBklM,grdR1fB4yP0Gzfq5bCS5sr-Qkzh0Dt1blyzTjvSZQntHdB...,DenSygeKamel69,5211,1710448215567,1223
0,eqX0OR2pT95kfsGGuvuGP8fsMVp0fD6BhroWPkFLLDEPq7A,t-NowNJoLsBbgIsGbxQkEfvRy97JFhx8b6NpQjACUo2PEQ,_0I0nSHVdqOYpg8NwOWRKftdAAfWaXyUcKmVRWAMKEU8PB...,Razørk Activoo,3166,1710416765119,626
0,Q-74eE2mC8N6DDd9DX_4ovZDugrWzemeW3RQoGQg_RIqB5DX,0uIonKasNQb1UuiWQ1rwmBH8u-MqOMxcnTFv_KLvBK3mm4I,1hEqg2b6R5ityhv7EIQpRTERDlxCVm9p4MV5ac6q1mpT9x...,NAAYIL,5992,1710434515875,904
0,r_BFQxSk4MyEbjNRSwttWI2MYHyCt0bvRtvQCwLNYwQc5O...,9toAjTK5hFECTx1xO6rsVBbj2XpCq0vlZcJ1-3bXuCODa6...,Zp4BKvdf1RJguhSTAD4-vUqWXIQg0Rh3AfIHncONyQZtM1...,SUPdoss,29,1710418897868,153
0,2anEu3NaPjTM5pOdOiPfzDf5OHNneE8Rz-nKXh8IOM4X8cyc,wCUnt66Lv1MNnBU7Qt_V6mVjrr53KEf7kIaNkIKqiQp6H1...,i99znuyReXuyqP1Z_nkJTv3RSGoJf1TH1COfR1BNBPkVkW...,gimme resources,3018,1710357916000,141
0,AaFuSnt-zl6HAbM9d6QZqjZ1VzAC_VtoyUVaTlsk1ys-OxI,jbbSaSzGpNy-ZDrb1-VVnSDjSlpMj_KmuK2yYIanIKYNVO4,9ItUibycAwSPebqeSuuYjbnaOVBq-UwBs9V02-cFeBuFHh...,Canam,6551,1710440963505,721


In [112]:
def get_matchids_by_puuid(df, api_key):

    matchids_list = []

    for i in range(len(df)):

        api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=20&api_key={}'.format(df['puuid'].iloc[i], api_key)

        resp = requests.get(api_url) # Sending a get request to the api_url
        print(resp.json())

        if resp.status_code == 200:
            matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
            matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 100 second') #approximate 110 second wait
                    time.sleep(10)

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=20&api_key={}'.format(df['puuid'].iloc[i], api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                    if resp.status_code == 200:
                        matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
                        matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
                        print('limit cost resolve')
                        break
                else:
                    break

    matchids_df = pd.concat(matchids_list) # Using pd.concat to concatenate the results stored in the "matchids_list"
    
    return matchids_df


In [117]:
matchids_df  = get_matchids_by_puuid(summoner_df, api_key)

['EUW1_6855752385', 'EUW1_6855680767', 'EUW1_6855611584', 'EUW1_6855581739', 'EUW1_6855540190', 'EUW1_6854557787', 'EUW1_6854447817', 'EUW1_6853858464', 'EUW1_6853477295', 'EUW1_6853419549', 'EUW1_6853304491', 'EUW1_6853235526', 'EUW1_6853201196', 'EUW1_6852266100', 'EUW1_6852150741', 'EUW1_6848214867', 'EUW1_6844074612', 'EUW1_6843966841', 'EUW1_6843916237', 'EUW1_6843818735']
['EUW1_6855105577', 'EUW1_6854897814', 'EUW1_6854847033', 'EUW1_6850544466', 'EUW1_6850454859', 'EUW1_6850408752', 'EUW1_6846312072', 'EUW1_6846291536', 'EUW1_6846247952', 'EUW1_6846215041', 'EUW1_6845622881', 'EUW1_6845537933', 'EUW1_6845483039', 'EUW1_6845446461', 'EUW1_6844917488', 'EUW1_6844525515', 'EUW1_6844463207', 'EUW1_6844359258', 'EUW1_6843791366', 'EUW1_6843749846']
['EUW1_6856135818', 'EUW1_6856103120', 'EUW1_6856068053', 'EUW1_6856021111', 'EUW1_6855968952', 'EUW1_6855911029', 'EUW1_6855866151', 'EUW1_6855842815', 'EUW1_6854626145', 'EUW1_6854586758', 'EUW1_6854537914', 'EUW1_6853631211', 'EUW1_685

In [228]:
matchids_df.reset_index(inplace=True)
matchids_df.drop(columns = 'index', inplace = True )
matchids_df

,matchid
0,EUW1_6855752385
1,EUW1_6855680767
2,EUW1_6855611584
3,EUW1_6855581739
4,EUW1_6855540190
...,...
195,EUW1_6851932907
196,EUW1_6850641070
197,EUW1_6850558780
198,EUW1_6850506016


In [229]:
matchids_df.to_csv('matchids.csv', index = False)

In [237]:
def get_match_info_by_matchId(match_ids, api_key):

    matchid_info_list = [] # List that will store the dataframes generated, for each match id in the for loop 

    for i in range(len(match_ids)):# len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        api_ulr = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # here we use the iloc function as explained above to form the api link for each matchid
        resp = requests.get(api_ulr) # Sending a get request to the api_url
        print(resp.status_code) # Printing the status_code of the request

        if resp.status_code == 200: # Here we need to use if statment, because of the riot API rate limit which is 20 requests every 1 sec and 100 requests every 2 minutes. A 200 status_code means that the request was successful so it can move on.
            pass                        
        elif resp.status_code == 429: # A 429 means that we have exceeded the rate limit. To solve that we need to wait and then try again to get a successful request
            while True: # To do taht we use a while true loop, to execute the if, elif statment repeatdly until the status_code is 200. After that we use break statement to come out of this loop.
                if resp.status_code == 429: 
                    print('429 delay try 10 second') #approximate 110 second wait to work
                    time.sleep(10) # Sleep

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # Trying the request again
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        # Same process we did in the beginning             
        matchid_info = pd.json_normalize(resp.json()['info']['participants'])
        matchid_info_list.append(matchid_info) # Merging all the dataframes generated and then stored in the matchid_info_list with the concat() function
        
    matchid_info_df = pd.concat(matchid_info_list)

    return matchid_info_df

In [238]:
matchs_info_df = get_match_info_by_matchId(matchids_df[:10], api_key)

200
200
200
200
200
200
200
200
200
200


In [239]:
matchs_info_df
matchs_info_df.to_csv('matchs_info.csv', index = False)


In [247]:
#print(matchs_info_df.columns.tolist())
"""
    Retire les colonnes inutiles (jugé par un expert de league of legends (Emilio))
"""
matchs_info_df = pd.read_csv('matchs_info.csv')
toRemove = ['championId', 'championName', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'inhibitorsLost','item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'nexusKills','nexusLost','nexusTakedowns','participantId','placement','playerAugment1','playerAugment2','playerAugment3','playerAugment4', 'playerScore0', 'playerScore1', 'playerScore3', 'playerScore4', 'playerScore5', 'playerScore6', 'playerScore7', 'playerScore8', 'playerScore9', 'playerScore10', 'playerScore11', 'playerSubteamId', 'profileIcon', 'puuid', 'riotIdGameName', 'riotIdTagline', 'role', 'subteamPlacement', 'summonerName', 'teamEarlySurrendered', 'teamId', 'teamPosition', 'challenges.dancedWithRiftHerald', 'challenges.killsOnRecentlyHealedByAramPack', 'challenges.killsWithHelpFromEpicMonster', 'challenges.lostAnInhibitor', 'challenges.outnumberedNexusKill', 'challenges.poroExplosions', 'challenges.snowballsHit', 'challenges.unseenRecalls','missions.playerScore0','missions.playerScore1','missions.playerScore2','missions.playerScore3','missions.playerScore4','missions.playerScore5','missions.playerScore6','missions.playerScore7','missions.playerScore8','missions.playerScore9','missions.playerScore10','missions.playerScore11', 'perks.statPerks.defense', 'perks.statPerks.flex', 'perks.statPerks.offense', 'perks.styles', 'challenges.teleportTakedowns', 'challenges.highestCrowdControlScore', 'challenges.earliestBaron', 'eligibleForProgression', 'gameEndedInEarlySurrender', 'lane', 'largestCriticalStrike', 'nexusKills', 'nexusLost', 'nexusTakedowns','individualPosition', 'summonerId','challenges.legendaryItemUsed','challenges.moreEnemyJungleThanOpponent', 'challenges.legendaryItemUsed', 'summonerId', 'individualPosition']
matchs_info_df_non_filtre = matchs_info_df
print(matchs_info_df_non_filtre.columns)


Index(['allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings',
       'bountyLevel', 'champExperience', 'champLevel', 'championId',
       'championName',
       ...
       'challenges.highestChampionDamage',
       'challenges.highestCrowdControlScore', 'challenges.earliestBaron',
       'challenges.soloTurretsLategame',
       'challenges.fasterSupportQuestCompletion',
       'challenges.highestWardKills',
       'challenges.baronBuffGoldAdvantageOverThreshold',
       'challenges.fastestLegendary', 'challenges.thirdInhibitorDestroyedTime',
       'challenges.hadAfkTeammate'],
      dtype='object', length=283)


In [248]:
matchs_info_df = matchs_info_df.drop(columns=toRemove, axis=1)

print(matchs_info_df.columns)


Index(['allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings',
       'bountyLevel', 'champExperience', 'champLevel', 'championTransform',
       'commandPings',
       ...
       'challenges.controlWardTimeCoverageInRiverOrEnemyHalf',
       'challenges.shortestTimeToAceFromFirstTakedown',
       'challenges.highestChampionDamage', 'challenges.soloTurretsLategame',
       'challenges.fasterSupportQuestCompletion',
       'challenges.highestWardKills',
       'challenges.baronBuffGoldAdvantageOverThreshold',
       'challenges.fastestLegendary', 'challenges.thirdInhibitorDestroyedTime',
       'challenges.hadAfkTeammate'],
      dtype='object', length=206)


In [257]:
matchs_info_test = matchs_info_df.groupby(np.arange(len(matchs_info_df))//5).sum()
pd.set_option('display.max_columns', None)

matchs_info_test

,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,getBackPings,goldEarned,goldSpent,holdPings,inhibitorKills,inhibitorTakedowns,itemsPurchased,killingSprees,kills,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,objectivesStolen,objectivesStolenAssists,onMyWayPings,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerScore2,pushPings,quadraKills,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerLevel,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.firstTurretKilledTime,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsUnderOwnTurret,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,challenges.maxLevelLeadLaneOpponent,challenges.mejaisFullStackInTime,challenges.multiKillOneSpell,challenges.multiTurretRiftHeraldCount,challenges.multikills,challenges.multikillsAfterAggressiveFlash,challenges.outerTurretExecutesBefore10Minutes,challenges.outnumberedKills,challenges.perfectDragonSoulsTaken,challenges.perfectGame,challenges.pickKillWithAlly,challenges.playedChampSelectPosition,challenges.quickCleanse,challenges.quickFirstTurret,challenges.quickSoloKills,challenges.riftHeraldTakedowns,challenges.saveAllyFromDeath,challenges.scuttleCrabKills,challenges.skillshotsDodged,challenges.skillshotsHit,challenges.soloBaronKills,challenges.soloKills,challenges.stealthWards

In [265]:
#toutes les deux lignes sont représentées sous la forme de parametre_0 et parametre_1 pour les deux équipes, on double le nombre de colonnes et on divise les lignes par deux
#pour avoir une ligne par match

#0 2 4 6 8 10 12 14 16 18
equipes_1 = matchs_info_test.iloc[::2]
equipes_1.reset_index(inplace=True)


equipes_2 = matchs_info_test.iloc[1::2]
equipes_2.reset_index(inplace=True)

In [264]:
equipes_1

,index,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,getBackPings,goldEarned,goldSpent,holdPings,inhibitorKills,inhibitorTakedowns,itemsPurchased,killingSprees,kills,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,objectivesStolen,objectivesStolenAssists,onMyWayPings,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerScore2,pushPings,quadraKills,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerLevel,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.firstTurretKilledTime,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsUnderOwnTurret,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,challenges.maxLevelLeadLaneOpponent,challenges.mejaisFullStackInTime,challenges.multiKillOneSpell,challenges.multiTurretRiftHeraldCount,challenges.multikills,challenges.multikillsAfterAggressiveFlash,challenges.outerTurretExecutesBefore10Minutes,challenges.outnumberedKills,challenges.perfectDragonSoulsTaken,challenges.perfectGame,challenges.pickKillWithAlly,challenges.playedChampSelectPosition,challenges.quickCleanse,challenges.quickFirstTurret,challenges.quickSoloKills,challenges.riftHeraldTakedowns,challenges.saveAllyFromDeath,challenges.scuttleCrabKills,challenges.skillshotsDodged,challenges.skillshotsHit,challenges.soloBaronKills,challenges.soloKills,challenges.stealt

In [266]:
equipes_2

,index,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,getBackPings,goldEarned,goldSpent,holdPings,inhibitorKills,inhibitorTakedowns,itemsPurchased,killingSprees,kills,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,objectivesStolen,objectivesStolenAssists,onMyWayPings,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerScore2,pushPings,quadraKills,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerLevel,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.firstTurretKilledTime,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsUnderOwnTurret,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,challenges.maxLevelLeadLaneOpponent,challenges.mejaisFullStackInTime,challenges.multiKillOneSpell,challenges.multiTurretRiftHeraldCount,challenges.multikills,challenges.multikillsAfterAggressiveFlash,challenges.outerTurretExecutesBefore10Minutes,challenges.outnumberedKills,challenges.perfectDragonSoulsTaken,challenges.perfectGame,challenges.pickKillWithAlly,challenges.playedChampSelectPosition,challenges.quickCleanse,challenges.quickFirstTurret,challenges.quickSoloKills,challenges.riftHeraldTakedowns,challenges.saveAllyFromDeath,challenges.scuttleCrabKills,challenges.skillshotsDodged,challenges.skillshotsHit,challenges.soloBaronKills,challenges.soloKills,challenges.stealt